In [1]:
import datetime
import pandas as pd
import numpy as np
import argparse
import sys

sys.path.append('../..')
from utils.func import norm_time

In [28]:
parser = argparse.ArgumentParser()
parser.add_argument('--train_date', type=int, default=2014)
parser.add_argument('--test_date', type=int, default=2015)
parser.add_argument('--code', type=list, default=['7203'])
# parser.add_argument('--code', type=list, default=['7203', '9984'])
# parser.add_argument('--code', type=list, default=['8301', '7203', '9501', '6758', '9984',
#                                                  '8306', '8411', '6501', '6752', '6502'])

args = parser.parse_args(args=[])

code_list = [str(x) for x in args.code]

# テキストを取り出す

In [21]:
for i, date in enumerate(range(2013, 2018)):
    tmp = pd.read_csv('../../data/news/' + str(date) + '.csv', encoding='cp932')
    tmp = tmp[tmp['Company_IDs(TSE)'].isin(code_list)]
    # tmp = tmp[tmp['Company_Relevance'] == str(100)]
    tmp = tmp[['Time_Stamp_Original(JST)', 
                    'Company_Code(TSE)', 
                    'Headline', 
                    'News_Source',
                    'Company_Relevance', 
                    'Keyword_Article']]

    # 欠損除去
    tmp = tmp[~tmp["Keyword_Article"].isnull()]

    # タグ除去
    tmp = tmp[(tmp['News_Source'] == '日経') | 
                 (tmp['News_Source'] == 'ＮＱＮ') |
                 (tmp['News_Source'] == 'ＱＵＩＣＫ') | 
                 (tmp['News_Source'] == 'Ｒ＆Ｉ')]

    tmp['code'] = tmp['Company_Code(TSE)'].astype(int)
    tmp['date'] = pd.to_datetime(tmp["Time_Stamp_Original(JST)"]).map(norm_time)
    tmp = tmp.set_index(['date', 'code'], drop=True)
    tmp = tmp.drop(['Time_Stamp_Original(JST)', 'News_Source', 'Company_Code(TSE)'], axis=1)

    if i == 0:
        df1 = tmp.copy()
    else:
        df1 = pd.concat([df1, tmp])

Headline Company_Relevance  \
date       code                                                               
2013-01-03 7203          <日経>◇大村・愛知知事「企業立地、着実に」　自動車２税撤廃要請                35   
2013-01-04 7203  <ＮＱＮ>◇日経平均、大幅続伸で始まる 「崖」回避と円安を好感し震災前水準上回る                36   
           7203            <日経>◇日経平均、大幅続伸で始まる 「崖」回避と円安を好感                36   
2013-01-05 7203          <ＮＱＮ>◇東証１部の時価総額、300兆円上回る　11年３月以来                36   
           7203       <ＮＱＮ>◇＜東証＞トヨタが一時５％超高　円安進行で一段の採算改善期待               100   

                                                   Keyword_Article  
date       code                                                     
2013-01-03 7203  愛知県:景気:対策:防災:減災:課題:活性化:自動車取得税:撤廃:国内生産:落ち込む:響く:...  
2013-01-04 7203  日経平均株価:続伸:前年:発生:３月:終値:水準:回復:円安:米国:歳出削減:財政:問題:回...  
           7203  日経平均株価:続伸:前年:上昇:発生:３月:終値:水準:回復:円安:米国:歳出削減:財政:問...  
2013-01-05 7203  時価総額:前年:増加:終値:発生:３月:休場:米国:財政:問題:回避:円安:ハイテク:買い:...  
           7203  大幅:年末年始:続伸:上昇:１月:高値:米国:財政:回避:円売り:外国為替市場:円相場:下落...

# 株価を取り出す

In [22]:
for i, code in enumerate(code_list):
    tmp = pd.read_csv('../../data/stock_price/' + str(code) + '.csv', index_col=0)
    tmp['code'] = int(code)
    if i == 0:
        df2 = tmp
    else:
        df2 = pd.concat([df2, tmp])

df2['date'] = pd.to_datetime(df2['date'])
df2 = df2.set_index(['date', 'code'], drop=True)
print(df2.shape)

(2175, 1)


# 形態素解析

In [30]:
import MeCab

In [31]:
m = MeCab.Tagger('-Ochasen')# ('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
soup = m.parse (text)
for row in soup.split("\n"):
    word =row.split("\t")[0]
    if word == "EOS":
        break
    else:
        pos = row.split("\t")[1]
        if len(word) > 1:
            output_words.append(word)

NameError: name 'text' is not defined

# テキストと株価をくっつける

In [23]:
# 時系列をくっつける
df3 = pd.concat([df1, df2], axis=1, join_axes=[df1.index], levels=[0, 1])
df3 = df3.sort_values(by=['code', 'date'])
# df3['Keyword_Article'] = \
# df3.groupby(level=[0,1]).apply(lambda x: ':<pad>:'.join(list(x['Keyword_Article'])))

df3 = df3.dropna()
df3 = df3[~df3.duplicated(subset=['Keyword_Article'])]
df3['price'] = \
     df3['adj_close'].groupby(level=['code']).pct_change(1).shift(-1)*100
df3 = df3.dropna()

/home/ts-zemi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  


In [27]:
df3[-10:]

Headline  \
date       code                                                     
2017-12-22 7203                     <日経>◇トヨタ、17年の世界販売、過去最高の1035万台   
           7203                       <日経>◇トヨタ労組、「ベア3000円」要求　５年連続   
2017-12-26 7203                         <日経>◇道産子ＥＶ「ネイクル」、２人乗りを開発へ   
2017-12-27 7203                        <日経>◇プリウスＰＨＶに特別仕様車　安全機能を装備   
           7203                    <日経>◇「アルファード」「ヴェルファイア」一部改良　トヨタ   
           7203                        <日経>◇ＡＩの富だれのもの？　配分や権限の契約指針   
           7203                    <日経>◇ウーバー、リース事業売却へ　上場へ不採算事業を整理   
2017-12-28 7203  <NQN>◇トヨタ11月生産、国内7.6％増の29万5631台　海外5.8％減の50万6931台   
           7203                      <R&I>ダイハツ工業の発行体格付AA-[安定的]を維持   
2017-12-29 7203                       <日経>◇トヨタ、11月の輸出7･8％増　中国向け好調   

                Company_Relevance  \
date       code                     
2017-12-22 7203               100   
           7203               100   
2017-12-26 7203                32   
2017-12-27 7203               100   
           7203               100   
           7203                26   
           7203                35   
2017-12-28 7203               100   
           7203                50   
2017-12-29 7203               100   

                                                   Keyword_Article  adj_close  \
date       code                                                                 
2017-12-22 7203  生産:販売実績:全体:台数:過去最高:新型車:販売:好調:措置:延長:受け:中国:小型車:伸...     7282.0   
           7203  労働:組合:ベースアップ:ベア:連続:自動車産業:先行き:不透明:生産:向上:判断:産業界:...     7282.0   
2017-12-26 7203  中小:北海道:本部:コンソーシアム:寒冷地:電気自動車:ＥＶ:開発:バイク:ウレタン:外装:...     7253.0   
2017-12-27 7203          装備:１月:アクセルペダル:スイッチ:駐車スペース:判断:全国:ペット:販売:価格     7274.0   
           7203  改良:１月:自転車:夜間:検知:カメラ:性能:向上:販売:目指す:価格:逸脱:避ける:操作:...     7274.0   
           7203  人工知能:ＡＩ:生み出す:利益:権限:契約:問題:ビッグデータ:提供:開発:安心:環境:不可...     7274.0   
           7203  シリコンバレー:最大手:運転:ベンチャー:売却:合意:確保:優遇:提供:不採算事業:整理:補...     7274.0   
2017-12-28 7203          １１月:生産:前年同月比:増:連続:増加:輸出:海外:減:国内生産:海外生産:世界     7221.0   
           7203  格付:維持:発行体格付:安定的:理由:廉価:小型車:トップ:シェア:海外:マレーシア:高い:...     7221.0   
2017-12-29 7203  １１月:生産:前年同月比:増:連続:前年:高級車:販売:好調:中国:東南アジア:中南米:北米...     7213.0   

                    price  
date       code            
2017-12-22 7203  0.000000  
           7203 -0.398242  
2017-12-26 7203  0.289535  
2017-12-27 7203  0.000000  
           7203  0.000000  
           7203  0.000000  
           7203 -0.728622  
2017-12-28 7203  0.000000  
           7203 -0.110788  
2017-12-29 7203  0.000000